In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import json
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout
from tensorflow.keras.utils import plot_model

In [3]:
import json

with open(r'C:\Users\91630\Desktop\college_intents.json') as f:
    data = json.load(f)

df = pd.DataFrame(data['intents'])
df.head()

,intent,text,responses
0,greeting,"[Hi, Hello, Hey, Good day, How are you?]","[Hello!, Good to see you!, Hi there, how can I..."
1,farewell,"[Goodbye, Bye, See you later, Talk to you later]","[Sad to see you go :(, Goodbye!, Come back soon!]"
2,creator,"[Who created you?, Who is your developer?, Who...",[I was created by Strydo Technologies PVt.LTD.]
3,identity,"[What is your name?, What should I call you?, ...",[You can call me Mind Reader. I'm a Chatbot.]
4,communication,[How can I communicate with the university or ...,[You can communicate with the university throu...


In [5]:
dic = {"intent":[],"text":[], "responses":[]}
for i in range (len(df)):
    ptrns = df[df.index==i]['text'].values[0]
    rspns = df[df.index==i]['responses'].values[0]
    tag = df[df.index==i]['intent'].values[0]
    for j in range(len(ptrns)):
        dic['intent'].append(tag)
        dic['text'].append(ptrns[j])
        dic['responses'].append(rspns)
df = pd.DataFrame.from_dict(dic)
df

,intent,text,responses
0,greeting,Hi,"[Hello!, Good to see you!, Hi there, how can I..."
1,greeting,Hello,"[Hello!, Good to see you!, Hi there, how can I..."
2,greeting,Hey,"[Hello!, Good to see you!, Hi there, how can I..."
3,greeting,Good day,"[Hello!, Good to see you!, Hi there, how can I..."
4,greeting,How are you?,"[Hello!, Good to see you!, Hi there, how can I..."
...,...,...,...
1051,transportation_options,What are the available options for transportat...,[The college may provide shuttle services for ...
1052,transportation_options,Does the college provide transportation services?,[The college may provide shuttle services for ...
1053,transportation_options,Are there public transportation options near t...,[The college may provide shuttle services for ...
1054,transportation_options,Is it easy to get around the campus and surrou...,[The college may provide shuttle services for ...


In [7]:
df['intent'].unique()

array(['greeting', 'farewell', 'creator', 'identity', 'communication',
       'casual_greeting', 'good_morning', 'good_afternoon',
       'good_evening', 'thank_you', 'sorry', 'hours', 'contact',
       'courses', 'fees', 'location', 'hostel', 'events', 'admission',
       'library', 'facilities', 'placement', 'school_choice', 'happiness',
       'strengths', 'complaints', 'improvements', 'accessibility',
       'academics_majors', 'academics_reputation', 'favorite_class',
       'professors_teachers', 'professors_office_hours',
       'professors_vs_TAs', 'first_choice_classes', 'class_format',
       'reading_writing', 'collaborative_work', 'academic_advising',
       'tutoring_programs', 'class_size', 'independent_study',
       'professor_research', 'study_abroad', 'first_year_experience',
       'typical_day', 'study_time', 'non_class_activities',
       'weekend_activities', 'social_scene', 'hometown_activities',
       'food', 'on_campus_housing', 'dorm_quality', 'popular_clubs'

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(lower=True, split=' ')
tokenizer.fit_on_texts(df['text'])
tokenizer.get_config()

{'num_words': None,
 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
 'lower': True,
 'split': ' ',
 'char_level': False,
 'oov_token': None,
 'document_count': 1056,
 'word_counts': '{"hi": 1, "hello": 1, "hey": 1, "good": 8, "day": 3, "how": 207, "are": 333, "you": 90, "goodbye": 1, "bye": 1, "see": 2, "later": 3, "talk": 1, "to": 202, "who": 21, "created": 1, "is": 217, "your": 70, "developer": 1, "made": 3, "what": 275, "name": 1, "should": 5, "i": 22, "call": 1, "introduce": 1, "yourself": 1, "can": 110, "communicate": 1, "with": 48, "the": 373, "university": 5, "or": 76, "contact": 3, "my": 20, "ward": 1, "communication": 1, "channels": 1, "for": 188, "students": 301, "and": 52, "parents": 7, "what\'s": 12, "up": 1, "doing": 1, "morning": 2, "afternoon": 2, "evening": 2, "thank": 1, "thanks": 1, "sorry": 1, "apologies": 1, "college": 185, "timings": 2, "when": 10, "open": 1, "hours": 6, "of": 134, "operation": 1, "telephone": 1, "number": 8, "get": 25, "courses": 18, "offered"

In [11]:
vocab_size = len(tokenizer.word_index)
print('number of unique words = ', vocab_size)

number of unique words =  1047


In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

ptrn2seq = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['intent'])
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))

X shape =  (1056, 21)
y shape =  (1056,)
num of classes =  219


In [15]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout
from tensorflow.keras.utils import plot_model

model = Sequential()
# Initialize the Sequential model

# Add Input layer
model.add(Input(shape=(X.shape[1],)))

# Add Embedding layer
model.add(Embedding(input_dim=vocab_size + 1, output_dim=100, mask_zero=True))

# Add LSTM and LayerNormalization layers
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())

model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())

model.add(LSTM(32))
model.add(LayerNormalization())

# Add Dense, LayerNormalization, and Dropout layers
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))

model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))

# Output layer with the number of unique labels in y
model.add(Dense(len(np.unique(y)), activation="softmax"))

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 21, 100)             │         104,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 21, 32)              │          17,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization                  │ (None, 21, 32)              │              64 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 21, 32)              │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_1                │ (None, 21, 32)              │              64 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 32)                  │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_2                │ (None, 32)                  │              64 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_3                │ (None, 128)                 │             256 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_4                │ (None, 128)                 │             256 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 219)                 │          28,251 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 188,155 (734.98 KB)

 Trainable params: 188,155 (734.98 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [21]:
model_history = model.fit(x=X,
                          y=y,
                          batch_size=10,
                          callbacks=[tensorflow.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)],
                          epochs=50)

Epoch 1/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.0137 - loss: 5.7958
Epoch 2/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.0568 - loss: 5.0945
Epoch 3/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.1337 - loss: 4.3488
Epoch 4/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2521 - loss: 3.7054
Epoch 5/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.3210 - loss: 3.1181
Epoch 6/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.4810 - loss: 2.3840
Epoch 7/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.5045 - loss: 2.0491
Epoch 8/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.6437 - loss: 1.6707
Epoch 9/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7437 - loss: 1.1843
Epoch 10/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7611 - loss: 1.0338
Epoch 11/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8244 - loss: 0.7999
Epoch 12/50
106/106 ━━━━━━━━━━━━━━━━━━━━

In [23]:
import re
import random

In [25]:
def generate_answer(pattern): 
    text = []
    txt = re.sub('[^a-zA-Z\']', ' ', pattern)
    txt = txt.lower()
    txt = txt.split()
    txt = " ".join(txt)
    text.append(txt)
        
    x_test = tokenizer.texts_to_sequences(text)
    x_test = np.array(x_test).squeeze()
    x_test = pad_sequences([x_test], padding='post', maxlen=X.shape[1])
    y_pred = model.predict(x_test)
    y_pred = y_pred.argmax()
    tag = lbl_enc.inverse_transform([y_pred])[0]
    responses = df[df['intent'] == tag]['responses'].values[0]

    print("you: {}".format(pattern))
    print("Chitti The Chatbot: {}".format(random.choice(responses)))

In [27]:
generate_answer("who is your creator")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 590ms/step
you: who is your creator
Chitti The Chatbot: I was created by Strydo Technologies PVt.LTD.


In [29]:
# Save the model
model.save('education_chatbot_model1.h5')

# Save the tokenizer
with open('education_tokenizer1.json', 'w') as f:
    json.dump(tokenizer.get_config(), f)

# Save the label encoder
with open('education_label_encoder1.json', 'w') as f:
    json.dump(lbl_enc.get_params(), f)
